In [29]:
# Import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy 
import random
import requests
from config import weather_api_key
from datetime import datetime
import json
import time
import gmaps
from config import g_key

In [5]:
#Avoid error when loading heatmaps from gmaps
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [30]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [3]:
#Import data from Weather Database
city_data_df=pd.read_csv('data/WeatherPy_challenge.csv')
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,Hilo,US,19.73,-155.09,78.80,61,1,4.70,clear sky,0.0,0.0
1,Te Anau,NZ,-45.42,167.72,46.06,79,3,3.09,clear sky,0.0,0.0
2,Agadez,NE,19.75,10.25,88.11,16,5,7.25,clear sky,0.0,0.0
3,Byron Bay,AU,-28.65,153.62,75.20,53,29,10.29,scattered clouds,0.0,0.0
4,Hobart,AU,-42.88,147.33,59.00,54,75,5.82,broken clouds,0.0,0.0


In [4]:
city_data_df.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)          float64
dtype: object

In [22]:
#Filter dataframe for maximum and minimum temperatures
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
is_raining=str(input("Do you want it to be raining? (yes/no) "))
is_snowing=str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) no


In [23]:
print(is_raining,is_snowing)

yes no


In [24]:
#Set values to yes and no
if is_raining=='yes':
    is_raining_count=0.01
else:
    is_raining_count=10000

if is_snowing=='yes':
    is_snowing_count=0.01
else:
    is_snowing_count=10000
print(is_raining_count,is_snowing_count)

0.01 10000


In [25]:
#Set dataframe based on criteria
preferred_cities_df=city_data_df.loc[(city_data_df['Max Temp']<= max_temp)&\
                                     (city_data_df['Max Temp']>=min_temp)&\
                                    (city_data_df['Rain (inches)']>=is_raining_count)|\
                                    (city_data_df['Snow (inches)']>=is_snowing_count)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
18,Kendari,ID,-3.94,122.50,78.91,88,100,3.94,moderate rain,0.2,0.0
189,Karimganj,IN,24.87,92.35,87.04,74,59,1.23,light rain,0.1,0.0
276,Pemangkat,ID,1.17,108.97,81.59,76,79,4.94,broken clouds,0.1,0.0
278,Afzalpur,IN,17.20,76.35,88.02,47,99,5.28,overcast clouds,0.1,0.0
285,San Rafael Del Sur,NI,11.85,-86.44,76.35,85,100,1.79,overcast clouds,0.1,0.0
286,Maracaibo,VE,10.63,-71.64,78.17,80,98,1.54,overcast clouds,0.1,0.0
332,Yellapur,IN,14.97,74.72,84.20,70,40,5.82,scattered clouds,0.1,0.0
373,Pontianak,ID,-0.03,109.33,80.64,86,99,2.98,light rain,0.1,0.0
436,Dafeng,CN,23.45,108.59,88.79,63,96,12.44,light rain,0.1,0.0


In [26]:
#Create dataframe to add the hotel name
hotel_df=preferred_cities_df[['City','Country','Max Temp','Current Description','Lat','Lng']]
hotel_df['Hotel Name']=''
hotel_df.head(10)

C:\Users\gamad\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
18,Kendari,ID,78.91,moderate rain,-3.94,122.50,
189,Karimganj,IN,87.04,light rain,24.87,92.35,
276,Pemangkat,ID,81.59,broken clouds,1.17,108.97,
278,Afzalpur,IN,88.02,overcast clouds,17.20,76.35,
285,San Rafael Del Sur,NI,76.35,overcast clouds,11.85,-86.44,
286,Maracaibo,VE,78.17,overcast clouds,10.63,-71.64,
332,Yellapur,IN,84.20,scattered clouds,14.97,74.72,
373,Pontianak,ID,80.64,light rain,-0.03,109.33,
436,Dafeng,CN,88.79,light rain,23.45,108.59,


In [31]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found ... skipping")

hotel_df.head(10)

C:\Users\gamad\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
18,Kendari,ID,78.91,moderate rain,-3.94,122.50,Hotel Wixel
189,Karimganj,IN,87.04,light rain,24.87,92.35,Ahar Hotel And Aheli Restaurant
276,Pemangkat,ID,81.59,broken clouds,1.17,108.97,Kost H.Syamsuddin
278,Afzalpur,IN,88.02,overcast clouds,17.20,76.35,SKG HOTELS
285,San Rafael Del Sur,NI,76.35,overcast clouds,11.85,-86.44,Hospedaje El Chino
286,Maracaibo,VE,78.17,overcast clouds,10.63,-71.64,Motel Aladdin
332,Yellapur,IN,84.20,scattered clouds,14.97,74.72,Gramdevi Ink's
373,Pontianak,ID,80.64,light rain,-0.03,109.33,Aston Pontianak Hotel & Convention Center
436,Dafeng,CN,88.79,light rain,23.45,108.59,Shenglong Hotel


In [35]:
# Create map with pop-up marker
#Define locations
locations=hotel_df[['Lat','Lng']]
#assign the figure variable
fig=gmaps.figure(center=(30.0,15.0),zoom_level=1.3)

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
#add the layers
fig.add_layer(marker_layer)
#call the figure to plot
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
# Create the output file (CSV).
output_data_file='data/WeatherPy_vacation.csv'
# Export the City_data into a CSV
city_data_df.to_csv(output_data_file,index=False)